In [1]:
import hsfs
from hops import hdfs
import boto3
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
62,application_1646303173729_0021,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
connection = hsfs.connection()
fs = connection.get_feature_store()

# Storage connector to s3
sc = fs.get_storage_connector("experiment-s3")

Connected. Call `.close()` to terminate connection gracefully.

In [3]:
sc.prepare_spark()

In [4]:
spark.sql("use {}".format(hdfs.project_name()))

DataFrame[]

In [5]:
s3_directory = "s3a://" + sc.bucket + "/axel_experiments"

In [6]:
def save(experiment_id, left: DataFrame, right: DataFrame):
    directory = s3_directory + "/bucketed-raw/{}".format(experiment_id)
    left.write.option("path", directory + "/left").format("parquet").bucketBy(10, "id").saveAsTable("left", mode = "overwrite")
    right.write.option("path", directory + "/right").format("parquet").bucketBy(10, "id").saveAsTable("right", mode = "overwrite")
    

In [7]:
s3_raw_directory = "s3a://" + sc.bucket + "/axel_experiments/raw/"
def load_data(experiment_directory: str):
    left_raw = s3_raw_directory + "/{}/left.parquet".format(experiment_directory)
    right_raw = s3_raw_directory + "/{}/right.parquet".format(experiment_directory)
    
    return spark.read.parquet(left_raw), spark.read.parquet(right_raw)

In [8]:
client = boto3.client(
    "s3",
    aws_access_key_id=sc._access_key,
    aws_secret_access_key=sc._secret_key,
)

In [9]:
folders_results = client.list_objects(Bucket=sc.bucket, Prefix="axel_experiments/raw/", Delimiter='/')

In [10]:
experiment_ids = []

for folder in folders_results.get("CommonPrefixes"):
    folder_name = folder.get("Prefix")
    experiment_ids.append(folder_name.split("/")[2])
experiment_ids

['10000-1_year', '100000-1_year', '1000000-1_year', '10000000-1_year']

In [11]:
def handle_experiments(ids):
    for experiment_id in ids:
        left_raw, right_raw = load_data(experiment_id)
        save(experiment_id, left_raw, right_raw)
        

In [12]:
handle_experiments(experiment_ids)